
# Assignment-2
### Pooja Bandal
### 2022-03-08




### Cloning the repository and setting up the notebook for import

In [ ]:
!rm -rf /content/Assignment_2
!git clone https://github.com/bandpooja/Assignment_2.git

Cloning into 'Assignment_2'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 28 (delta 3), reused 24 (delta 1), pack-reused 0
Unpacking objects: 100% (28/28), done.


In [ ]:
import sys
import os
os.listdir('.')

['.config', 'Assignment_2', 'sample_data']

In [ ]:
sys.path.append('/content/Assignment_2')


I need to install nltk and pytrec-eval-terrier libraries again in the notebook because I used a different environment.


In [ ]:
!pip install pytrec-eval-terrier

     |████████████████████████████████| 287 kB 5.2 MB/s 



### Importing modules

Importing all the modules used in the project.

In [ ]:
import concurrent
import json
import nltk
from nltk.corpus import brown
import pytrec_eval
from tqdm import tqdm

from utils.preprocess import cleansing, cleansing_test, preprocess_pipeline
from utils.ngram_model import NGramModel
from utils.helper import load_test


### Fitting and saving the models

Saving and fitting the models.

In [ ]:
nltk.download('brown')
nltk.download('punkt')

# region Model Fitting
new_list = brown.sents()
min_freq = 15
tokenized_sent = preprocess_pipeline(new_list)
final_train, vocabulary = cleansing(tokenized_sent, min_freq)

ns = [1, 2, 3, 5, 10]
model_loc = 'models'

for n in ns:
    model = NGramModel(n=n, model_loc=model_loc, vocabulary=vocabulary)
    model.fit(final_train)
    model.save_model()
# endregion

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### Validating the model

Checking if it performs in a test case.

In [ ]:
# region validation
previous_tokens = ["the", "jury"]

for n in ns:
    model = NGramModel(n=n, model_loc=model_loc, vocabulary=vocabulary)
    model.load_model()
    print(f"{n}-gram model prediction: {model.get_suggestions(previous_tokens)}")
# endregion

1-gram model prediction: {1: [('<unk>', 0.09079413723563301)], 5: [('<unk>', 0.09079413723563301), ('the', 0.05431409351795359), (',', 0.045282573629980144), ('<s>', 0.04450944137313257), ('<e>', 0.04450944137313257)], 10: [('<unk>', 0.09079413723563301), ('the', 0.05431409351795359), (',', 0.045282573629980144), ('<s>', 0.04450944137313257), ('<e>', 0.04450944137313257), ('.', 0.039005919584930444), ('of', 0.028264349132865418), ('and', 0.02239677209520394), ('to', 0.020304812825922596), ('a', 0.018016465543606676)]}
2-gram model prediction: {1: [('.', 0.0015676438313215238)], 5: [('.', 0.0015676438313215238), (',', 0.0015676438313215238), ('said', 0.0014108794481893713), ('<unk>', 0.0009405862987929142), ('of', 0.0007838219156607619)], 10: [('.', 0.0015676438313215238), (',', 0.0015676438313215238), ('said', 0.0014108794481893713), ('<unk>', 0.0009405862987929142), ('of', 0.0007838219156607619), ('that', 0.0007838219156607619), ('and', 0.0004702931493964571), ('was', 0.00047029314939

### Testing

Making predictions on all the data in `APPLING1DAT.643` file.

In [ ]:
# region testing
# Making prediction on all the birdbeck data Just checking success at k for all the predictions
test_df = load_test(file_loc='/content/Assignment_2/data/APPLING1DAT.643')
tokenized_sent = preprocess_pipeline(test_df['previous-tokens'].values.tolist(), remove_empty=False)
final_test = cleansing_test(tokenized_sent, vocabulary)
test_df['final-test'] = final_test
ns = [1,2]
queries = [{} for _ in ns]
results_eval = [{} for _ in ns]

for idx, n in enumerate(ns):
    model = NGramModel(n=n, model_loc=model_loc, vocabulary=vocabulary)
    model.load_model()

    argument_list = final_test
    suggestions = []
    with concurrent.futures.ProcessPoolExecutor() as executor:
        for result in executor.map(model.get_suggestions, argument_list):
            suggestions.append(result)

    query = queries[idx]
    result_eval = results_eval[idx]
    for fill_in_word, test_previous_tokens, suggestion in tqdm(zip(test_df['fill-in-word'], final_test,
                                                                    suggestions), total=len(final_test),
                                                                desc=f'Predictions-{n}-gram-model'):
        query[f"{' '.join(test_previous_tokens)} *"] = {fill_in_word: 1}
        result_eval[f"{' '.join(test_previous_tokens)} *"] = {}

        for word in [w[0] for w in suggestion[1]]:
            result_eval[f"{' '.join(test_previous_tokens)} *"][word] = 1

        for word in [w[0] for w in suggestion[5]]:
            if word not in result_eval[f"{' '.join(test_previous_tokens)} *"].keys():
                result_eval[f"{' '.join(test_previous_tokens)} *"][word] = 1 / 5

        for word in [w[0] for w in suggestion[10]]:
            if word not in result_eval[f"{' '.join(test_previous_tokens)} *"].keys():
                result_eval[f"{' '.join(test_previous_tokens)} *"][word] = 1 / 10

    print('#' * 20)
    print(f'Stats of {n}-gram model')
    print('#' * 20)
    evaluator = pytrec_eval.RelevanceEvaluator(query, {'success'})

    # print(json.dumps(evaluator.evaluate(result_eval), indent=1))
    eval = evaluator.evaluate(result_eval)

    for measure in sorted(list(eval[list(eval.keys())[0]].keys())):
        print(measure, 'average:',
              pytrec_eval.compute_aggregated_measure(
                  measure, [query_measures[measure] for query_measures in eval.values()])
              )
    print('#' * 20)
# endregion


Predictions-1-gram-model: 100%|██████████| 198/198 [00:00<00:00, 44805.62it/s]


####################
Stats of 1-gram model
####################
success_1 average: 0.0
success_10 average: 0.0
success_5 average: 0.0
####################


Predictions-2-gram-model: 100%|██████████| 198/198 [00:00<00:00, 25089.79it/s]


####################
Stats of 2-gram model
####################
success_1 average: 0.007407407407407408
success_10 average: 0.007407407407407408
success_5 average: 0.007407407407407408
####################


Predictions-3-gram-model: 100%|██████████| 198/198 [00:00<00:00, 25089.79it/s]


####################
Stats of 3-gram model
####################
success_1 average: 0.002222222222222223
success_10 average: 0.002222222222222223
success_5 average: 0.002222222222222223
####################


Predictions-5-gram-model: 100%|██████████| 198/198 [00:00<00:00, 25089.79it/s]


####################
Stats of 5-gram model
####################
success_1 average: 0.007407407407407408
success_10 average: 0.014814814814814815
success_5 average: 0.007407407407407408
####################


Predictions-10-gram-model: 100%|██████████| 198/198 [00:00<00:00, 25089.79it/s]


####################
Stats of 10-gram model
####################
success_1 average: 0.007407407407407408
success_10 average: 0.014814814814814815
success_5 average: 0.007407407407407408
####################


In [ ]:
# END